In [1]:
import numpy as np

In [2]:
observed_np = np.random.rand(2,1)
observed_np

array([[0.01198856],
       [0.28742471]])

In [3]:
def K(H:np.array, P:np.array, R:np.array):
    return P*H.T*(H*P*H.T + R)**-1

In [4]:
def Xn(H:np.array, Xhat:np.array, K:np.array, z:np.array): # Xhat is the observed X
    return Xhat + K * (z - H * Xhat)

In [5]:
class P:
    def __init__(self, A,X) -> None:
        self.value = self.multiply(A,X)
    def update(self, H, K):
        self.value = self.value - K*H*self.value
        return self.value
    def multiply(self,A,X):
        self.value = A*X*A.T
        return self.value

In [6]:
class Kalman:
    def __init__(self, X, P0, delta) -> None:
        self.X = X
        self.P = P(P0)
        self.Xpred = None
        self.delta = delta
    def predict(self, Fk, Bk, uk, Qk):
        
        return self.Xpred, self.P 

In [7]:
def create_F(delta):
    return np.array([[1, delta], [0, 1]])


# Bk is the constant that we multiply acceleration by and delta itself: [delta**2 / 2, delta]

In [8]:
class B_vector(object):
    def __init__(self, delta, dimensions=1) -> None:
            self.value = np.array([delta**2/2] + ([delta] * dimensions))

# Uk is just the acceleration vector

# Qk is untracked noise on the covariance matrix

# H is a matrix of sensor noise

# Rk is the covariance of the uncertainty of a sensor reading. its mean is equal to the sensor's reading

# for any possible reading,  we have two gaussian distributions: 
### 1: probability that our sensor reading Zk is a (mis)measurement and 
### 2: the certainty that our previous estimate thinks that Zk is the reading we should see

# our predicted measurement (based on position and velocity) is (Mu0, Sigma0) = (Hk*Xkpred, Hk*Pk*Hk)
# our observed measurement is (Mu1, Sigma1) = (Zk, Rk)

# Divide by Hk (remember that K includes H, so K changes to K1) to get:
#### K1 = Pk * Hk.T * (Hk * Pk * Hk.T + Rk)**-1
### Xkhat = Xkpredk + K1 * (Zk - HkXpredk)
### Pkhat = Pk - K1 * Hk * Pk

# ^ this is the complete setup for the update step

## forward pass

In [9]:
position = np.array([0,0,0])
velocity = np.array([1,2,0])
assert len(position) == len(velocity)
delta = 3
# F0 = np.array([
#     [1, 0, delta, 0],
#     [0, 1, 0, delta],
#     [0, 0, 1, 0],
#     [0, 0, 0, 1]
# ])
dims = len(position) + len(velocity)
F0 = np.eye((dims))
for row in range(int(dims/2)):
    F0[row, len(position) + row] = delta

x0 = np.append(position, velocity).T
np.matmul(F0, x0)

array([3., 6., 0., 1., 2., 0.])

In [10]:
def F_matrix(x:np.array, delta):
    """x is position and velocity in an n by 1 vector"""
    dims = max(x.shape)
    F0 = np.eye((dims))
    for row in range(int(dims/2)):
        F0[row, len(position) + row] = delta
    return F0

In [11]:
def create_x_F_P(position:list, velocity:list, delta, P0):
    x = np.array([position + velocity])
    F = F_matrix(x, delta)
    P = np.matmul(F, P0, F.T)
    return x, F, P

In [15]:
def forward(x:np.array,F:np.array,P:np.array, H:np.array, R:np.array, z:np.array):
    K1 = np.matmul(P, H.T, np.linalg.inv(np.matmul(H, P, H.T) + R))
    xk = x + np.matmul(K, (z - np.matmul(H, x)))
    Pk = P - np.matmul(K, H, P)
    return xk, K1, Pk

In [13]:
position = [0,0,0]
velocity = [1,2,0]
delta = 3
p0 = np.eye(len(position) + len(velocity))
x, F, P = create_x_F_P(position, velocity, delta, p0)
forward(x,F,P)